## TODO:   
Load Raven  
Load Freebird   
Load Avianz   
Load BC Zenodo  
Anqa to Selections  
Itegrate Voice Detect  
Integrate Animal Sound Object Detect

## Reformat Bird Soundscapes & Training Data to Anqa

Converts a dog's breakfast of sources, annotation programs and methodologies into a cleaned and unified `.csv` data format *Anqa* with time-frequency bounding boxes and a separate `.csv` file for per-recording metadata.

### Motivation & Summary

* Creates a dataset of soundscapes with a specified maximum length, in `.flac` audio format.
* `metadata.csv` file contains with any metadata available, with one row per soundscape.
* `annotations.csv`, for future processing 
* Optional conversion to Raven-visualisable `.selections.txt` files

This notebook is intended to demonstrate one step in a ML model training pipeline, to unify data from various sources.  The immediate next step could be human-in-loop review using Raven in which case the updated Raven Selections files would need to be re-run through it to update the underlying Anqa labels csv.  Alternatively another method or a custom notebook could be used to directly work with the Anqa labels.  The step after that would likely be to use the annotated bounding boxes to crop the soundscapes into ML-training short-crop format.

For validation and testing steps, the data from those splits could be left in their 1-minute soundscapes to more accurately reflect their real-life use-case.

The formating schema is trying to separate metadata from annotations. Arbitrary extra columns such as 'reviewed by' can be added to the metadata `.csv`, whilst keeping the annotations in a simple standardised `.csv`, or in `.selections` files.

### Methods

**Source Data (Labels plus corresponding .wav, .ogg or .flac files)**
* A previously compiled Anqa dataset (To change or standardise the recording lengths, or update detection proposals)
* Raven `.selections` tables
* A BirdCLEF - Zenodo style soundscape `.csv` tidytable with one bounding box per row, as per [here](https://zenodo.org/records/7018484)
* AviaNZ `.data` labels - see note below 
* Freebird `.tag` xml labels
* An existing BirdCLEF-style metadata dataframe with data such as licenses, lat, long, reviewed-by, reviewed-on, etc.

If starting from scratch, people supplying the audio should be encouraged supply the metadata in `.csv`, one row per recording, and for annotation to use Raven with e-bird and iNaturalist codes, and mark bounding boxes (frequency and time bounds) on every bird-call, including 'unknown' on any calls they can not identify.  One minute of annotations per recording is optimal (randomly sampled from a longer recording if need be).

**Notes for using AviaNZ to label source material**
* Any annotation with multiple birds will be excluded, including anything with the tui/bellbird merged label
* Annotations must have both time and frequency bounds
* Every bird call should have a bounding box.  Please add 'unknown' for any instances of a bird-call that can not be identified rather than using combinations of birds
* Annotations must be human-labelled, not labelled with methods built-in to AviaNZ or other ML models.

**Intermediate formatting step (Automated)**  

Parses the source files, and creates three dataframes:

1. Metadata, from the existing one if it exists, otherwise a new (mostly empty) one is created
2. Labels.  These will only be used from their original 'source' format, for example `.selections` from Raven.
3. Invalid rows.  Where the matching files are missing or can't be opened.

**Propose missing labels (Optional)**  

* Delete any annotations previously labelled 'h_detect' or 'a_detect'
* Run through every sample, and detect and label any human voice.  Label 'h_detect'
* Run through every sample and detect other animal or bird sounds.  Label 'a_detect'

**Crop longer recordings to a fixed maximum length (Optional)**
Restrict each file to a maximum length (default 60 seconds) chosen from the whole with the most identified labels.  If none found then a random period will be chosen.  The point of doing this is to maximise the value of the labelling person's time.  15 different files of one minute each is much more diversity than a single 15-minute file with same birds in the same background noise.

**Save in Anqa format**  

* (Optional), re-save the recordings in `.flac` format
* Save the Annotatioins 
* Save the metadata

**Human-in-loop editing**
* Create an updated raven dataset with `.selections` files, plus `annotations.csv` and `metadata.csv`
* Externally review the `.selections.txt` files in raven, then re-run the process above.
* The main reasons to do this would be to change the length of the soundscapes, or take advantage of improved detection/cleaning models.
* Precedence will be given to the existing human-tagged labels over anything like automated bird or voice detection.
* Previously auto-labelled annotations will be over-written.

**Re-Running an existing Anqa Dataset**
* The main reasons to do this would be to change the length of the soundscapes, or take advantage of improved detection/cleaning models.
* Precedence will be given to the existing human-tagged labels over anything like automated bird or voice detection.
* Previously auto-labelled annotations will be over-written.

### Notebook Configuration
(Adjust as needed)

In [ ]:
#Note: 'anqa_destn' must differ from source directory if saving new audio files
notebook_use = {'project_root': '/home/olly/Desktop/Kaytoo',
                'anqa_destn': '/home/olly/Desktop/Kaytoo/data/sample_label_formats/converted_to_anqa_twice', 
                'source_type': 'anqa', #any of: 'anqa', 'raven', 'birdclef', 'bc-zenodo', 'avianz', 'freebird'
                'debug': True,
                'max_seconds': 60,  #Segment and/or crop any data type to this maximum length
                'crop_method': 'keep_all', #any of 'keep_all', 'random',  in future: 'max_annotations', 'max_detections'
                'padding': None, # Any of 'pad', 'noise', 'zero', None
                'save_new_flac_files': True, # If true the original source will be re-written into .flac files of length max_seconds
                }

In [2]:
anqua_data = {'audio_files': '/home/olly/Desktop/Kaytoo/data/sample_label_formats/converted_to_anqa/audio',
              'metadata': '/home/olly/Desktop/Kaytoo/data/sample_label_formats/converted_to_anqa/metadata.csv',
              'annotations': '/home/olly/Desktop/Kaytoo/data/sample_label_formats/converted_to_anqa/annotations.csv',
              'naming_csv': '/home/olly/Desktop/Kaytoo/data/sample_label_formats/birdclef/metadata/naming.csv',
              'naming_cols' : {'source_name': 'eBird','save_name': 'eBird'},
             }

raven_data = {'audio_files': '/home/olly/Desktop/Kaytoo/data/sample_label_formats/raven_kokako_olly',
              'naming_csv': '/home/olly/Desktop/Kaytoo/data/experiments/exp_2/exp_2_deploy/exp_2_bird_map.csv',
              'naming_cols' : {'source_name': 'eBird','save_name': 'eBird'},
             }

avianz_data = {'audio_files': '/home/olly/Desktop/Kaytoo/data/sample_label_formats/avianz_janzoon_brian',
               'naming_csv': "filepath_needed",
               'naming_cols' : {'source_name': 'eBird','save_name': 'eBird'},
              }

birdclef_data = {
                'audio_files': '/home/olly/Desktop/Kaytoo/data/sample_label_formats/bc25_audio_files',
                'metadata': '/home/olly/Desktop/Kaytoo/data/sample_label_formats/birdclef/metadata/train.csv',
                'naming_csv': '/home/olly/Desktop/Kaytoo/data/sample_label_formats/birdclef/metadata/naming.csv',
                'naming_cols' : {'source_name': 'eBird','save_name': 'eBird'},
                }

#Post BirdCLEF annotated soundscapes on zenodo.org:   https://zenodo.org/search?q=birdclef&l=list&p=1&s=10&sort=bestmatch
bc_zenodo_data = {'audio_files': '/home/olly/Desktop/Kaytoo/data/sample_label_formats/zenodo_birdclef/Sierra_Nevada/',
                  'metadata': {'lat': 37.0,
                               'long': -118.5, 
                               'licence': 'some_licence',
                               'url' : 'some_web_address',
                               'collection' : 'zenodo',
                               'author' : 'Mr Bigglesworth'
                               },
                   'annotations': '/home/olly/Desktop/Kaytoo/data/sample_label_formats/zenodo_birdclef/Sierra_Nevada/annotations.csv',
                   'naming_csv': '/home/olly/Desktop/Kaytoo/data/sample_label_formats/birdclef/metadata/naming.csv',
                   'naming_cols' : {'source_name': 'eBird','save_name': 'eBird'},
                  }

freebird_data = {'audio_files': '/home/olly/Desktop/Kaytoo/data/sample_label_formats/freebird_tier_1',
                 'naming_csv': '/home/olly/Desktop/Kaytoo/data/sample_label_formats/freebird_species_list_05112013.csv',
                 'naming_cols' : {'source_name': 'Code', 'save_name': 'eBird'}
                }

data_sources = {'anqa' : anqua_data,
                'raven' : raven_data,
                'avianz' : avianz_data,
                'birdclef' : birdclef_data,
                'bc_zenodo' : bc_zenodo_data,
                'freebird' : freebird_data
                }

In [3]:
import os, sys
from dotenv import load_dotenv
import random
import numpy as np
import pandas as pd
import torch
from pathlib import Path
from tqdm.auto import tqdm
from IPython.display import display
import ipywidgets as widgets
button = widgets.Button(description="Continue")
output = widgets.Output()

#%matplotlib widget  
torch.set_num_threads(1)

In [4]:
class Config():
    def __init__(self, options: dict, data_sources: dict):
        self.SOURCE_DATA_TYPE = options['source_type']
        self.DEBUG = options['debug']
        self.MAX_SECONDS = options['max_seconds']
        self.CROP_METHOD = options['crop_method']
        self.PADDING = options['padding']
        self.SAVE_NEW_FLAC_FILES = options['save_new_flac_files']
        self.SAVE_RAVEN_SELECTIONS = False # User must manually change at end of the notebook if wanted
        self.SOURCE_NAME = self.SAVE_NAME = None
        _naming_cols = data_sources[self.SOURCE_DATA_TYPE].get('naming_cols')
        if _naming_cols is not None:
            self.SOURCE_NAME = _naming_cols.get('source_name')
            self.SAVE_NAME = _naming_cols.get('save_name')


class FilePaths():
    def __init__(self, notebook_use: dict, data_sources: dict):
        _data_type = notebook_use['source_type']
        _source_cfg = data_sources[_data_type]
        _metadata = _source_cfg.get('metadata')
        _naming = _source_cfg.get('naming_csv')
        _annotations = _source_cfg.get('annotations')
        
        self.PROJECT_ROOT = Path(notebook_use['project_root'])
        self.AUDIO_FILES = Path(data_sources[_data_type]['audio_files'])
        self.METADATA = Path(_metadata) if _metadata is not None else None
        self.NAMING_CSV = Path(_naming) if _naming is not None else None
        self.ANNOTATION_SRC = Path(_annotations) if _annotations is not None else None

        # Output directory (ensure it exists)
        self.ANQA_DESTN = Path(notebook_use.get('anqa_destn'))
        self.ANQA_DESTN.mkdir(parents=True, exist_ok=True)

        self.ANNOTATIONS_DESTN = self.ANQA_DESTN / 'annotations.csv'
        self.METADATA_DESTN = self.ANQA_DESTN / 'metadata.csv'

config = Config(notebook_use, data_sources)
paths = FilePaths(notebook_use, data_sources)

In [5]:
load_dotenv(dotenv_path= paths.PROJECT_ROOT / ".env")

for p in os.getenv("PYTHONPATH", "").split(os.pathsep):
    if p:
        print(f' Adding Python path: {p}')
        sys.path.append(str(Path(p).resolve()))

 Adding Python path: ../../wildpytools/src
 Adding Python path: ./src


In [6]:
from wildpytools.audio import (load_from_anqa,
                               load_from_raven,
                               load_from_freebird,
                               load_from_birdclef,
                               load_from_bc_zenodo,
                               load_from_avianz,
                               anqa_to_raven_selections,
                               ToAnqa)

In [7]:
class VoiceDetector():
    def __init__(self, chunk_len, threshold=0.1, no_voice=0, voice=20):
        model, (get_speech_timestamps, _, read_audio, _, _) = torch.hub.load(repo_or_dir='snakers4/silero-vad',
                                                                 model='silero_vad', verbose=False)
        self.model=model
        self.chunk_len=chunk_len
        self.threshold=threshold
        self.get_stamps = get_speech_timestamps

    def detect(self, np_wav):
        speech_timestamps = self.get_stamps(torch.Tensor(np_wav), self.model, threshold=self.threshold)
        voice_detect = np.zeros_like(np_wav)
        for st in speech_timestamps:
            voice_detect[st['start']: st['end']] = 20
    
        #downsample to match the power plot axis
        voice_detect = np.pad(voice_detect, 
                            (0, int(np.ceil(len(voice_detect) / self.chunk_len) * self.chunk_len - len(voice_detect))))
        voice_detect = voice_detect.reshape((-1, self.chunk_len)).max(axis=1)  # Use max to preserve speech detection
        return voice_detect
    
def calc_signal_pwr(wav, chunk_len, sr=32000):
    power = wav ** 2 
    power = np.pad(power, (0, int(np.ceil(len(power) / chunk_len) * chunk_len - len(power))))
    power = power.reshape((-1, chunk_len)).sum(axis=1)
    return power


def detect_voice(labels: pd.DataFrame,
                 metadata: pd.DataFrame,
                 threshold: float = 0.5,
                 ):
    """Run a voice detection model over the spectrogram to try and locate any human-voice

    Args:
        df_labels (_type_): _description_
        df_meta (_type_): _description_
    """
    return labels, metadata


def detect_animal_sound(labels: pd.DataFrame,
                        metadata: pd.DataFrame,
                        ):

    """Run an object detection model over spectrograms to propose any missing labels for bird or animal sounds

    Args:
        df_labels (_type_): _description_
        df_meta (_type_): _description_
    """
    return labels, metadata

## Load source data

In [8]:
if paths.METADATA is not None:
    meta_df = pd.read_csv(paths.METADATA)
    display(meta_df.head(3))

,primary_label,secondary_labels,type,filename,collection,rating,url,latitude,longitude,scientific_name,common_name,author,license,recorded_on,reviewed_by,reviewed_on,Filepath,source_file,source_start_s,source_end_s
0,1194042,[''],[''],CSA18802_from_0.flac,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,5.7892,-73.5504,Copiphora colombiae,Copiphora colombiae,Orlando Acevedo-Charry,cc-by-nc-sa 4.0,NaN,NaN,NaN,/home/olly/Desktop/Kaytoo/data/sample_label_fo...,1194042/CSA18802.ogg,0.0,46.0
1,135045,[''],[''],iNat1208550_from_0.flac,iNat,0.0,https://static.inaturalist.org/sounds/1208550.wav,6.0424,-75.2530,Andinobates opisthomelas,Andean Poison Frog,Unknown,cc-by-nc 4.0,NaN,NaN,NaN,/home/olly/Desktop/Kaytoo/data/sample_label_fo...,135045/iNat1208550.ogg,0.0,60.0
2,135045,[''],[''],iNat1208550_from_60.flac,iNat,0.0,https://static.inaturalist.org/sounds/1208550.wav,6.0424,-75.2530,Andinobates opisthomelas,Andean Poison Frog,Unknown,cc-by-nc 4.0,NaN,NaN,NaN,/home/olly/Desktop/Kaytoo/data/sample_label_fo...,135045/iNat1208550.ogg,60.0,120.0


Setup mapping between the source data and the new dataset

In [9]:
if paths.NAMING_CSV is not None:
    source_names = pd.read_csv(paths.NAMING_CSV)
    source_col = config.SOURCE_NAME
    save_col = config.SAVE_NAME
    save_name_map = dict(zip(source_names.loc[:, source_col], source_names.loc[:, save_col]))
    display(source_names.head(3))

,eBird,CommonName,ScientificName,Class
0,1139490,Ragoniella pulchella,Ragoniella pulchella,insecta
1,1192948,Oxyprora surinamensis,Oxyprora surinamensis,insecta
2,1194042,Copiphora colombiae,Copiphora colombiae,insecta


In [10]:
SOURCE_LOADERS = {
    'anqa': {
        'func': load_from_anqa,
        'kwargs': {'metadata_path': paths.METADATA,
                   'labels_path': paths.ANNOTATION_SRC},
    },
    'raven': {
        'func': load_from_raven,
        'kwargs': {'metadata_path': paths.METADATA},
    },
    'bc-zenodo': {
        'func': load_from_bc_zenodo,
        'kwargs': {'labels_path': paths.ANNOTATION_SRC,
                   'metadata': paths.METADATA},
    },
    'birdclef': {
        'func': load_from_birdclef,
        'kwargs': {'metadata_path': paths.METADATA},
    },
    'freebird': {
        'func': load_from_freebird,
        'kwargs': {},
    },
    'avianz': {
        'func': load_from_avianz,
        'kwargs': {},
    },
}

source_type = config.SOURCE_DATA_TYPE
print(f'Loading source data of the type {source_type}')
loader_cfg = SOURCE_LOADERS[source_type]
loader_func = loader_cfg['func']
loader_kwargs = loader_cfg.get('kwargs', {}).copy()
loader_kwargs['name_map'] = save_name_map

df_label, df_meta, df_invalid = loader_func(paths.AUDIO_FILES, **loader_kwargs)

if config.DEBUG:
    df_label = df_label.head(40)
    df_meta = df_meta.head(40)
display(df_label.head())

Loading source data of the type anqa
                    Filename    Label  Start Time (s)  End Time (s)  \
0       CSA18802_from_0.flac  1194042             0.0          46.0   
1    iNat1208550_from_0.flac   135045             0.0          60.0   
2   iNat1208550_from_60.flac   135045             0.0          60.0   
3  iNat1208550_from_120.flac   135045             0.0           9.0   
4       CSA18793_from_0.flac  1346504             0.0          60.0   

   Low Freq (Hz)  High Freq (Hz)  Avg Power Density (dB FS/Hz)  
0            0.0         16000.0                         -34.8  
1            0.0         16000.0                         -44.5  
2            0.0         16000.0                         -40.9  
3            0.0         16000.0                         -40.2  
4            0.0         16000.0                         -39.6  


,Filename,Label,Start Time (s),End Time (s),Low Freq (Hz),High Freq (Hz),Avg Power Density (dB FS/Hz)
0,CSA18802_from_0.flac,1194042,0.0,46.0,0.0,16000.0,-34.8
1,iNat1208550_from_0.flac,135045,0.0,60.0,0.0,16000.0,-44.5
2,iNat1208550_from_60.flac,135045,0.0,60.0,0.0,16000.0,-40.9
3,iNat1208550_from_120.flac,135045,0.0,9.0,0.0,16000.0,-40.2
4,CSA18793_from_0.flac,1346504,0.0,60.0,0.0,16000.0,-39.6


Note that we're ignoring any 'secondary' labels at this point, but if there is a primary/secondary labelling it is retained in the metadata

In [11]:
df_label.head(3)

,Filename,Label,Start Time (s),End Time (s),Low Freq (Hz),High Freq (Hz),Avg Power Density (dB FS/Hz)
0,CSA18802_from_0.flac,1194042,0.0,46.0,0.0,16000.0,-34.8
1,iNat1208550_from_0.flac,135045,0.0,60.0,0.0,16000.0,-44.5
2,iNat1208550_from_60.flac,135045,0.0,60.0,0.0,16000.0,-40.9


In [12]:
df_meta.head(3)

,primary_label,secondary_labels,type,filename,collection,rating,url,latitude,longitude,scientific_name,common_name,author,license,recorded_on,reviewed_by,reviewed_on,Filepath,source_file,source_start_s,source_end_s
0,1194042,[''],[''],CSA18802_from_0.flac,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,5.7892,-73.5504,Copiphora colombiae,Copiphora colombiae,Orlando Acevedo-Charry,cc-by-nc-sa 4.0,NaN,NaN,NaN,/home/olly/Desktop/Kaytoo/data/sample_label_fo...,1194042/CSA18802.ogg,0.0,46.0
1,135045,[''],[''],iNat1208550_from_0.flac,iNat,0.0,https://static.inaturalist.org/sounds/1208550.wav,6.0424,-75.2530,Andinobates opisthomelas,Andean Poison Frog,Unknown,cc-by-nc 4.0,NaN,NaN,NaN,/home/olly/Desktop/Kaytoo/data/sample_label_fo...,135045/iNat1208550.ogg,0.0,60.0
2,135045,[''],[''],iNat1208550_from_60.flac,iNat,0.0,https://static.inaturalist.org/sounds/1208550.wav,6.0424,-75.2530,Andinobates opisthomelas,Andean Poison Frog,Unknown,cc-by-nc 4.0,NaN,NaN,NaN,/home/olly/Desktop/Kaytoo/data/sample_label_fo...,135045/iNat1208550.ogg,60.0,120.0


In [13]:
df_meta.shape

(40, 20)

In [14]:
df_invalid.head()

""


In [15]:
df_invalid.shape

(0, 0)

## Pre-filter the audio files

* Voice detection
* Bird-call detection

In [16]:
df_label, df_meta = detect_voice(df_label, df_meta, threshold=False)
df_label, df_meta = detect_animal_sound(df_label, df_meta)

## Convert the source data to Anqa Format

In [17]:
df_meta.head()

,primary_label,secondary_labels,type,filename,collection,rating,url,latitude,longitude,scientific_name,common_name,author,license,recorded_on,reviewed_by,reviewed_on,Filepath,source_file,source_start_s,source_end_s
0,1194042,[''],[''],CSA18802_from_0.flac,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,5.7892,-73.5504,Copiphora colombiae,Copiphora colombiae,Orlando Acevedo-Charry,cc-by-nc-sa 4.0,NaN,NaN,NaN,/home/olly/Desktop/Kaytoo/data/sample_label_fo...,1194042/CSA18802.ogg,0.0,46.0
1,135045,[''],[''],iNat1208550_from_0.flac,iNat,0.0,https://static.inaturalist.org/sounds/1208550.wav,6.0424,-75.2530,Andinobates opisthomelas,Andean Poison Frog,Unknown,cc-by-nc 4.0,NaN,NaN,NaN,/home/olly/Desktop/Kaytoo/data/sample_label_fo...,135045/iNat1208550.ogg,0.0,60.0
2,135045,[''],[''],iNat1208550_from_60.flac,iNat,0.0,https://static.inaturalist.org/sounds/1208550.wav,6.0424,-75.2530,Andinobates opisthomelas,Andean Poison Frog,Unknown,cc-by-nc 4.0,NaN,NaN,NaN,/home/olly/Desktop/Kaytoo/data/sample_label_fo...,135045/iNat1208550.ogg,60.0,120.0
3,135045,[''],[''],iNat1208550_from_120.flac,iNat,0.0,https://static.inaturalist.org/sounds/1208550.wav,6.0424,-75.2530,Andinobates opisthomelas,Andean Poison Frog,Unknown,cc-by-nc 4.0,NaN,NaN,NaN,/home/olly/Desktop/Kaytoo/data/sample_label_fo...,135045/iNat1208550.ogg,120.0,129.0
4,1346504,[''],[''],CSA18793_from_0.flac,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,5.7892,-73.5504,Neoconocephalus brachypterus,Neoconocephalus brachypterus,Orlando Acevedo-Charry,cc-by-nc-sa 4.0,NaN,NaN,NaN,/home/olly/Desktop/Kaytoo/data/sample_label_fo...,1346504/CSA18793.ogg,0.0,60.0


In [18]:
converter = ToAnqa(
            source_dir=paths.AUDIO_FILES,
            destn_dir=paths.ANQA_DESTN,
            name_map=save_name_map,
            save_audio=config.SAVE_NEW_FLAC_FILES,
            max_seconds=config.MAX_SECONDS,
            end_padding=config.PADDING,
            crop_method=config.CROP_METHOD,
            n_jobs=4,
            )

df_labels, df_meta = converter.convert_all(df_label, df_meta)

Converting recordings:   0%|          | 0/40 [00:00<?, ?it/s]

Converting recordings: 100%|██████████| 40/40 [00:07<00:00,  5.64it/s]


In [19]:
df_labels.head()

,Filename,Label,Start Time (s),End Time (s),Low Freq (Hz),High Freq (Hz),Avg Power Density (dB FS/Hz)
0,CSA18802_from_0.flac,1194042,0.0,46.0,0.0,16000.0,-34.8
1,iNat1208550_from_0.flac,135045,0.0,60.0,0.0,16000.0,-44.5
2,iNat1208550_from_60.flac,135045,0.0,60.0,0.0,16000.0,-40.9
3,iNat1208550_from_120.flac,135045,0.0,9.0,0.0,16000.0,-40.2
4,CSA18793_from_0.flac,1346504,0.0,60.0,0.0,16000.0,-39.6


In [20]:
df_meta.head()

,primary_label,secondary_labels,type,filename,collection,rating,url,latitude,longitude,scientific_name,common_name,author,license,recorded_on,reviewed_by,reviewed_on,Filepath,source_file,source_start_s,source_end_s
0,1194042,[''],[''],CSA18802_from_0.flac,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,5.7892,-73.5504,Copiphora colombiae,Copiphora colombiae,Orlando Acevedo-Charry,cc-by-nc-sa 4.0,NaN,NaN,NaN,/home/olly/Desktop/Kaytoo/data/sample_label_fo...,1194042/CSA18802.ogg,0.0,46.0
1,135045,[''],[''],iNat1208550_from_0.flac,iNat,0.0,https://static.inaturalist.org/sounds/1208550.wav,6.0424,-75.2530,Andinobates opisthomelas,Andean Poison Frog,Unknown,cc-by-nc 4.0,NaN,NaN,NaN,/home/olly/Desktop/Kaytoo/data/sample_label_fo...,135045/iNat1208550.ogg,0.0,60.0
2,135045,[''],[''],iNat1208550_from_60.flac,iNat,0.0,https://static.inaturalist.org/sounds/1208550.wav,6.0424,-75.2530,Andinobates opisthomelas,Andean Poison Frog,Unknown,cc-by-nc 4.0,NaN,NaN,NaN,/home/olly/Desktop/Kaytoo/data/sample_label_fo...,135045/iNat1208550.ogg,60.0,120.0
3,135045,[''],[''],iNat1208550_from_120.flac,iNat,0.0,https://static.inaturalist.org/sounds/1208550.wav,6.0424,-75.2530,Andinobates opisthomelas,Andean Poison Frog,Unknown,cc-by-nc 4.0,NaN,NaN,NaN,/home/olly/Desktop/Kaytoo/data/sample_label_fo...,135045/iNat1208550.ogg,120.0,129.0
4,1346504,[''],[''],CSA18793_from_0.flac,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,5.7892,-73.5504,Neoconocephalus brachypterus,Neoconocephalus brachypterus,Orlando Acevedo-Charry,cc-by-nc-sa 4.0,NaN,NaN,NaN,/home/olly/Desktop/Kaytoo/data/sample_label_fo...,1346504/CSA18793.ogg,0.0,60.0


In [21]:
df_meta.to_csv(paths.METADATA_DESTN, index=False)
df_labels.to_csv(paths.ANNOTATIONS_DESTN, index=False)

### (Optional) Create `.selections` for viewing or editing in Raven
Re-run these cells manually to produce `.selections.txt` files for viewing in [Raven](https://www.ravensoundsoftware.com/)

In [22]:
if config.SAVE_RAVEN_SELECTIONS:
    anqa_to_raven_selections(df_labels, paths.ANQA_DESTN)  #paths.new_raven_dataset, df_labels
    selections = list(paths.ANQA_DESTN.rglob('*.selections.txt'))
    display(selections[:3])

In [23]:
if config.SAVE_RAVEN_SELECTIONS:
    random_file = random.choice(selections)
    selection = pd.read_csv(random_file, sep="\t")
    display(selection.head())

In [24]:
config.SAVE_RAVEN_SELECTIONS = True